In [1]:

import numpy as np
import chemtools
from iodata import load_one
from pyscf import gto, scf


np.random.seed(123)

# Create and dump inputs/outputs

In [2]:
from iodata import load_one, dump_one

mol = load_one('water_neutral.fchk')
dump_one(mol, 'water.molden')

# Try to load from regular fchk 

In [3]:
mol_pyscf = gto.Mole()
mol_pyscf.atom = 'water_neutral.fchk'
mol_pyscf.build()

NameError: name 'sp' is not defined

# Try to load from molden

In [4]:
# first line must be deleted, otherwise it the format is incorrect 
molden_file = 'water.molden'

def delete_first_line(text_file):
    with open(text_file, 'r+') as f:
        to_read = f.read().splitlines(True)
        to_write = ' '.join([i for i in to_read[1:]])
        f.seek(0)
        f.write(to_write)
        f.truncate()

delete_first_line('water.molden')

mol_pyscf = gto.Mole()
mol_pyscf.atom = molden_file
mol_pyscf.build()

NameError: name 'sp' is not defined

# Load manually from chemtools

In [5]:
mol = chemtools.Molecule.from_file('water_neutral.fchk')

def make_pyscf_input(numbers, coords):
    numbers = np.around(numbers, decimals=1)
    return list(zip(numbers, coords))

mol_pyscf = gto.M(
    atom = make_pyscf_input(mol.numbers, mol.coordinates),
    basis = mol.obasis_name
)
mf = scf.RHF(mol_pyscf).run()

grid_3d = np.random.random((1,3))
# test a calculation
mol_pyscf.eval_gto('GTOval_sph', grid_3d)

converged SCF energy = -75.6682970586362


array([[ 2.34768316e-02,  3.49956937e-01,  3.88254554e-01,
         2.11417418e-01,  6.02471985e-01,  2.41481133e-01,
         2.11776932e-01,  4.16710385e-01,  1.67024689e-01,
         1.46479254e-01,  1.23689031e-01,  4.95766907e-02,
         4.34783427e-02,  5.83055793e-01,  2.04951882e-01,
        -3.83615832e-01,  5.11335049e-01,  6.10484406e-01,
         1.65996075e-01,  5.83498328e-02, -1.09215487e-01,
         1.45577168e-01,  1.73805006e-01,  3.94485357e-01,
         2.39254465e-01, -7.17180427e-02, -2.49281981e-01,
        -1.78929554e-01,  2.50509679e-01,  1.79567126e-01,
         2.46511140e-05,  1.73626464e-02,  5.22619891e-02,
        -2.57118464e-06, -2.56325049e-05, -5.99109872e-06,
        -4.11320592e-03, -4.10051340e-02, -9.58415135e-03,
         2.33218841e-04,  5.43421533e-04, -6.04588911e-04,
         5.45103613e-05, -1.15079884e-03,  1.86280145e-06,
         8.40058541e-03,  4.15713167e-02, -5.63568359e-07,
         4.36854852e-07,  1.07161128e-06, -8.73319198e-0